In [1]:
#pip install findspark
#/usr/local/opt/apache-spark/libexec/sbin
#./start-thriftserver.sh

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark").config('spark.driver.memory', '32g').config("hive.server2.thrift.port", 10000).config("spark.sql.hive.thriftServer.singleSession", True).enableHiveSupport().getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/19 12:31:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType,TimestampType
from pyspark.sql.types import DateType
from pyspark.sql.functions import *

schema = StructType([
    StructField("ID", IntegerType()),
    StructField("Speed", DoubleType()),
    StructField("TravelTime", IntegerType()),
    StructField("Status", IntegerType()),
    StructField("timedate", StringType()),
    StructField("LinkId", IntegerType()),
    StructField("LinkPoints", StringType()),
    StructField("EncodedLinkPoints", StringType()),
    StructField("EncodedPolyLineLvls", StringType()),
    StructField("Owner", StringType()),
    StructField("TranscomId", IntegerType()),
    StructField("Borough", StringType()),
    StructField("Link_Name", StringType())
])

schema1 = "ID bigint, Speed float, TravelTime int, Status int, TimeDate string, LinkId int, LinkPoints string, EncodedLinkPoints string, EncodedPolyLineLvls string, Owner string, TranscomId int, Borough string,Link_Name string"
df2 = spark.read.format("csv").option("header", "true").schema(schema).load("file:/Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv")
df2.printSchema()
# .option("inferSchema", "true")

root
 |-- ID: integer (nullable = true)
 |-- Speed: double (nullable = true)
 |-- TravelTime: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- timedate: string (nullable = true)
 |-- LinkId: integer (nullable = true)
 |-- LinkPoints: string (nullable = true)
 |-- EncodedLinkPoints: string (nullable = true)
 |-- EncodedPolyLineLvls: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- TranscomId: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Link_Name: string (nullable = true)



In [4]:
df2 = df2.withColumn("timedate", to_timestamp("timedate", 'MM/dd/yyyy hh:mm:ss a'))
df2.na.drop()
df2.createOrReplaceTempView("dataset")

df2.count()
df2.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Speed: double (nullable = true)
 |-- TravelTime: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- timedate: timestamp (nullable = true)
 |-- LinkId: integer (nullable = true)
 |-- LinkPoints: string (nullable = true)
 |-- EncodedLinkPoints: string (nullable = true)
 |-- EncodedPolyLineLvls: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- TranscomId: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Link_Name: string (nullable = true)



In [8]:
year = '2017'
start = year+'-01-01'
end = year+'-12-31'
query = "select ID,Speed,TravelTime,Status,timedate,LinkId,LinkPoints,EncodedLinkPoints,EncodedPolyLineLvls,Owner,TranscomId,Borough,Link_Name from dataset"
where = "where timedate >= '"+start+"' and timedate <= '"+end+"' order by timedate asc"

In [9]:
y2017 = spark.sql(query + " "+where)

In [10]:
y2017.write.format('csv').mode('overwrite').save('file:/Users/minoseah629/Desktop/ait614/1year'+year+'.csv')

In [7]:
columns = "timedate, speed, traveltime"
select = "select "+columns #+",count(second(timedate)), dayofweek(timedate)"
from1 = " from dataset"
where1 = " where year(timedate) >= 2017"
groupby = " group by "+columns+", dayofweek(timedate)"
orderby = " order by "+columns
query = select + from1 +where1+  groupby +orderby


In [32]:
columns1= "year(timedate), month(timedate), day(timedate), hour(timedate), minute(timedate), second(timedate)"
select1 = "select "+columns1+", count(second(timedate))"
groupby2= " group by "+columns1
query1 = select1 + from1 + groupby2

In [33]:
df2.createOrReplaceTempView("dataset")
linkpoints = spark.sql(query1)
linkpoints.show()

22/03/13 00:30:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DATA_AS_OF
 Schema: timedate
Expected: timedate but found: DATA_AS_OF
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


+--------------+---------------+-------------+--------------+----------------+----------------+-----------------------+
|year(timedate)|month(timedate)|day(timedate)|hour(timedate)|minute(timedate)|second(timedate)|count(second(timedate))|
+--------------+---------------+-------------+--------------+----------------+----------------+-----------------------+
|          2018|             11|           17|            10|              52|              40|                      7|
|          2018|             11|           17|            11|              12|              43|                      7|
|          2018|             11|           17|            12|              48|              16|                      7|
|          2018|             11|           17|            12|              48|              40|                      6|
|          2019|              6|            6|             1|              53|               6|                      7|
|          2018|             11|        

In [34]:
linkpoints.write.format('csv').mode('overwrite').save('file:/Users/minoseah629/Desktop/ait614/second.csv')

22/03/13 00:31:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DATA_AS_OF
 Schema: timedate
Expected: timedate but found: DATA_AS_OF
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


In [36]:
df1.createOrReplaceTempView("noschema")
spark.sql("select max(length(_c6)) from noschema").show()

+----------------+
|max(length(_c6))|
+----------------+
|             255|
+----------------+



In [14]:
spark.stop()